# Api Twitter
* https://datascienceparichay.com/article/get-data-from-twitter-api-in-python-step-by-step-guide/
* https://developer.twitter.com/en/portal/projects/1391772372416532480/apps/20820453/keys

Análisis de sentimiento en Español:
* https://platzi.com/tutoriales/1874-python-lenguaje-natural/5654-realiza-un-analisis-de-sentimiento-en-3-pasos-con-python/
    * Ejemplo https://twitter.com/whaleandjaguar_?lang=en

## Tutorial de búsqueda
### Acceso para búsqueda avanzada
Para acceder como usario ver [aquí](https://www.geeksforgeeks.org/python-api-me-in-tweepy/)

In [1]:
import tweepy as tw
import getpass

In [2]:
my_api_key=getpass.getpass()

 ·························


In [4]:
my_api_secret=getpass.getpass()

 ··················································


In [5]:
auth = tw.OAuthHandler(my_api_key, my_api_secret)

In [6]:
api = tw.API(auth, wait_on_rate_limit=True)

In [9]:
search_query = "#SOSColomia OR #ParoNacional OR #Cali"

In [21]:
# get tweets from the API
tweets = tw.Cursor(api.search,
              q=search_query,
              lang="es",
              since="2021-03-01" #https://es.wikipedia.org/wiki/Protestas_en_Colombia_de_2021
                  ).items(5000) #empty for all 

In [22]:
# store the API responses in a list
tweets_copy = []
for tweet in tweets:
    tweets_copy.append(tweet)
    
print("Total Tweets fetched:", len(tweets_copy))

Total Tweets fetched: 5000


El módulo `pickle` permité salvar objetos en archivos binarios. De ésta manera podemos grabar la información completa sobre la búsqueda para un análisis posterior

In [130]:
#Save the list of tweet objects as a pickle file
import pickle
f=open('tweets_copy.pickle','wb')
pickle.dump(tweets_copy,f)
f.close()

In [131]:
!ls -lh tweets_copy.pickle

-rw-r--r-- 1 restrepo restrepo 25M May 13 13:36 tweets_copy.pickle


Cada elemento de la lista es un objeto tweet, que equivala a una fila de una base de datos no relacional. Su documentación está disponible [aquí](https://developer.twitter.com/en/docs/twitter-api/v1/data-dictionary/object-model/tweet).

Para acceder al primer hastashg de un tweet debemos caminar la estructura:

lista → diccionario → lista → diccionario

In [115]:
tweets_copy[1].entities['hashtags'][0]['text']

'Cali'

## Ejemplo de DataFrame con información básica

In [26]:
import pandas as pd

# intialize the dataframe
tweets_df = pd.DataFrame()

# populate the dataframe
for tweet in tweets_copy:
    hashtags = []
    try:
        for hashtag in tweet.entities["hashtags"]:
            hashtags.append(hashtag["text"])
        text = api.get_status(id=tweet.id, tweet_mode='extended').full_text
    except:
        pass
    tweets_df = tweets_df.append(pd.DataFrame({'user_name': tweet.user.name, 
                                               'user_location': tweet.user.location,\
                                               'user_description': tweet.user.description,
                                               'user_verified': tweet.user.verified,
                                               'date': tweet.created_at,
                                               'text': text, 
                                               'hashtags': [hashtags if hashtags else None],
                                               'source': tweet.source}))
    tweets_df = tweets_df.reset_index(drop=True)

# show the dataframe
tweets_df.head()

,user_name,user_location,user_description,user_verified,date,text,hashtags,source
0,Liliana Morales,,,False,2021-05-12 05:52:23,RT @SemanarioVOZ: El Semanario VOZ recalca su ...,None,Twitter for Android
1,Quique,"Bogotá, D.C., Colombia",Actitud positiva,False,2021-05-12 05:52:21,"RT @COMANDANTE_EJC: En #Cali, mi mensaje hacia...",[Cali],Twitter for Android
2,David Peña,,,False,2021-05-12 05:52:13,RT @photomauricio: En la décima jornada del Pa...,None,Twitter for Android
3,tulelegon,"Bogotá, D.C., Colombia",URIBISTA 💯 %,False,2021-05-12 05:52:12,RT @primeravoztv: Esto está pasando en estos m...,None,Twitter for iPhone
4,Aleyda Giraldo A,,,False,2021-05-12 05:52:09,RT @HRI_ONG: Vemos con preocupación las Fallas...,None,Twitter Web App


In [132]:
tweets_df.shape

(5000, 8)

In [34]:
#Save to standard JSON file
tweets_df.to_json('tweets_df.json',orient='records')

### Ejemplo de análisis de datos
Obtenga la lista de hashtags

In [156]:
str(
    #first five elements
    tweets_df["hashtags"].dropna().sum()[:5]
).replace("]",",...]")

"['Cali', 'Cali', 'Colombia', 'ParoNacional', 'Cali',...]"

In [ ]:
aparecen

In [149]:
len(tweets_df["hashtags"].dropna().sum())

6268

veces

Obtener lista de hashtags únicos

In [155]:
str(
 #first five elements
 list(set(tweets_df['hashtags'].dropna().sum()))[:5]
).replace(']',',...]')

"['usme', 'Asesinado', 'Valentía', 'ColombiaTierraDeAtletas', 'Caldas',...]"

In [ ]:
TOTAL:

In [151]:
len(  list(set(tweets_df['hashtags'].dropna().sum()))  )

352

__Actividad__: Organizar los hashtags en orden decendiente de acuerdo al número de veces que aparecen en la lista completa de los mismos

Hint:

In [128]:
l=[2,8,7,8,6,7,8]

In [129]:
l.count(8)

3

### Grabar el objeto para análisis posteriores

In [102]:
import pickle
f=open('tweets_copy.pickle','rb')
tw_cp=pickle.load(f)
f.close()

In [103]:
tweet=tw_cp[0]

# Full DATA:
El atributo `._json` nos permite acceder a la información no estructurado del objeto de tweet completo desglosado en: https://developer.twitter.com/en/docs/twitter-api/v1/data-dictionary/object-model/tweet


In [165]:
print(f'{str(tweet._json)[:76]}...')

{'created_at': 'Wed May 12 05:52:23 +0000 2021', 'id': 1392356965578420225, ...


Si se adiciona como la fila de un DataFrame podremos visuzalizar la base de datos no estructurada completa

In [76]:
full_df=pd.DataFrame()
for tweet in tweets_copy:
    full_df=full_df.append(tweet._json,ignore_index=True)

In [105]:
pd.set_option('display.max_columns',50)

In [107]:
full_df[:1]

,contributors,coordinates,created_at,entities,favorite_count,favorited,geo,id,id_str,in_reply_to_screen_name,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,is_quote_status,lang,metadata,place,retweet_count,retweeted,retweeted_status,source,text,truncated,user,extended_entities,possibly_sensitive,quoted_status,quoted_status_id,quoted_status_id_str
0,None,None,Wed May 12 05:52:23 +0000 2021,"{'hashtags': [], 'symbols': [], 'user_mentions...",0.0,0.0,None,1392356965578420224,1392356965578420225,None,None,None,None,None,0.0,es,"{'iso_language_code': 'es', 'result_type': 're...",None,77.0,0.0,{'created_at': 'Wed May 12 01:10:27 +0000 2021...,"<a href=""http://twitter.com/download/android"" ...",RT @SemanarioVOZ: El Semanario VOZ recalca su ...,0.0,"{'id': 1392150058087428102, 'id_str': '1392150...",NaN,NaN,NaN,NaN,NaN


In [92]:
full_df['id']=full_df['id'].astype(int)

In [95]:
full_df.to_json('full_df.json',orient='records')

_________